In [1]:
import torch
from torchsummary import summary
from torch.utils.data import random_split, DataLoader
from torchvision.datasets import CIFAR10, Food101
from torchvision.transforms import v2
from pathlib import Path
import wandb

import sys

sys.path.append("../src")

from trainer import Trainer
from module import VGGNetModule
from utils import model_size
from callbacks import OverfitCallback, EarlyStoppingCallback
from logger import WandbLogger
from dataset import MapDataset

In [2]:
data_path = Path("../data")
logs_path = Path("../logs")
logs_path.mkdir(exist_ok=True)

In [3]:
logger = WandbLogger(
    project_name="ImageClassification",
    config={
        "model_architecture": "VGGNet",
        "batch_size": 64,
        "max_epochs": 100,
        "optimizer": {
            "name": "Adam",
        },
        "train_split": 53025,
        "val_split": 22725
    },
    logs_path=logs_path
)

In [4]:
dataset = Food101(data_path, split="train", download=True)

train_dataset, val_dataset = random_split(
    dataset, [logger.config["train_split"], logger.config["val_split"]]
)

val_transforms = v2.Compose([
    # Normalize
    v2.ToImage(),
    v2.ToDtype(torch.float32, scale=True),
    v2.Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.2023, 0.1994, 0.2010))
])

train_dataset = MapDataset(train_dataset, transform=v2.Compose([
    # Data auguments
    v2.RandomCrop(size=(32, 32), padding=4, padding_mode='reflect'),
    v2.RandomHorizontalFlip(),

    val_transforms
]))

val_dataset = MapDataset(val_dataset, transform=val_transforms)

train_dataloader = DataLoader(
    train_dataset, batch_size=logger.config["batch_size"], shuffle=True)
val_dataloader = DataLoader(
    val_dataset, batch_size=logger.config["batch_size"])

In [5]:
callbacks = [
    EarlyStoppingCallback(min_val_accuracy=80.0, accuracy_diff=4.0, wait_epochs=5),
    # OverfitCallback(limit_batches=2, max_epochs=500),
]

In [6]:
module = VGGNetModule()

trainer = Trainer(
    module=module,
    logger=logger,
    callbacks=callbacks,
    logs_path=logs_path,
    fast_dev_run=False,
    measure_time=True
)

model_size(module.model)

model size: 21.01 MB


In [7]:
summary(module.model, input_size=(3, 32, 32))

RuntimeError: Given groups=1, weight of size [16, 16, 3, 3], expected input[2, 8, 32, 32] to have 16 channels, but got 8 channels instead

In [7]:
module.device

'cpu'

In [ ]:
try:
    trainer.fit(train_dataloader, val_dataloader)
except KeyboardInterrupt as e:
    print("Run stopped!")
finally:
    wandb.finish()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: sampath017. Use `wandb login --relogin` to force relogin


Time per epoch: 133.57 seconds
Epoch: 0, train_accuracy: 34.33, val_accuracy: 44.65
Epoch: 1, train_accuracy: 46.38, val_accuracy: 50.94
Epoch: 2, train_accuracy: 50.58, val_accuracy: 53.21
Epoch: 3, train_accuracy: 53.21, val_accuracy: 56.15
Epoch: 4, train_accuracy: 55.05, val_accuracy: 56.74
Epoch: 5, train_accuracy: 56.20, val_accuracy: 59.15
Epoch: 6, train_accuracy: 57.33, val_accuracy: 60.12
Epoch: 7, train_accuracy: 58.38, val_accuracy: 59.91
Epoch: 8, train_accuracy: 58.85, val_accuracy: 60.36
Epoch: 9, train_accuracy: 59.34, val_accuracy: 61.41
Epoch: 10, train_accuracy: 60.29, val_accuracy: 61.84
Epoch: 11, train_accuracy: 60.77, val_accuracy: 62.74
Epoch: 12, train_accuracy: 60.86, val_accuracy: 62.47
Epoch: 13, train_accuracy: 61.58, val_accuracy: 62.06
Epoch: 14, train_accuracy: 62.08, val_accuracy: 63.44
Epoch: 15, train_accuracy: 62.24, val_accuracy: 64.00
Epoch: 16, train_accuracy: 62.60, val_accuracy: 60.72


[Metrics](https://api.wandb.ai/links/sampath017/iwrrziwg)